In [1]:
from web3.middleware import geth_poa_middleware
from web3 import Web3
import json
import requests

## Static data
mainnet = '' # get yourself an ankr api key
testnet = 'https://data-seed-prebsc-1-s1.binance.org:8545/'

# Public keys
main_addr = ''
test_addr = ''
robot_addr = ''

# Mainnet vars
bscscan_url = 'https://api.bscscan.com/api?module=contract&action=getabi&address='

# Contracts
egg_addr = '0xf952fc3ca7325cc27d15885d37117676d25bfda6'
incubator_addr = '0xA0349A63bC48987B941a91A4d156c05B6bfbd321' # layer 2
#incubator_addr = '0x22cC57c9ec341152834F216289A1824d61b47855' # layer 1

# Fetch EGG ABI from BSCSCAN
response = requests.get(bscscan_url+egg_addr).json()
egg_abi = json.loads(response['result'])

# Fetch incubator ABI from BSCSCAN
response = requests.get(bscscan_url+incubator_addr).json()
incubator_abi = json.loads(response['result'])

# Robot account
with open('robot_account', 'r') as file:
    robot_account = file.read()

# Pool info keys
keys = ('lpToken',
'allocPoint',
'lastRewardBlock',
'accGoosePerShare',
'depositFeeBP',
'maxDepositAmount',
'currentDepositAmount')

In [ ]:
# Connect to network
w3 = Web3(Web3.HTTPProvider(mainnet))
w3.middleware_onion.inject(geth_poa_middleware, layer=0)

print('is connected?',w3.isConnected())

In [6]:
## One off data pulls
# Incubator contract and pool info
inc_contract = w3.eth.contract(Web3.toChecksumAddress(incubator_addr), abi=incubator_abi)

# Create EGG contract on mainnet
egg_contract = w3.eth.contract(Web3.toChecksumAddress(egg_addr), abi=egg_abi)

In [7]:
## Dynamic calls

print('Balances')
print('BNB', float(w3.fromWei(w3.eth.get_balance(robot_addr), 'ether')))

# Fetch EGG balance and convert to major units
balance_egg = float(w3.fromWei(egg_contract.functions.balanceOf(robot_addr).call(), 'ether'))
print('EGG', balance_egg)

Balances
BNB 0.19136875
EGG 0.0


In [9]:
## Incubator pool info
"""
Each incubator has multiple pools, we are only interested in restricted pools.
In layer 1 and 2 the highest yielding pool (at inception) was 0.

Loop thru all pools

for i in range(0,50):
    print(i, inc_contract.functions.poolInfo(i).call())
"""

pool_info = dict(zip(keys,inc_contract.functions.poolInfo(0).call()))
available_deposit = pool_info['maxDepositAmount'] - pool_info['currentDepositAmount']
available_deposit = float(w3.fromWei(available_deposit, 'ether'))

print('available_deposit', available_deposit)
pool_info

available_deposit 0.03741731277203745


{'lpToken': '0xF952Fc3ca7325Cc27D15885d37117676d25BfdA6',
 'allocPoint': 4000,
 'lastRewardBlock': 5612203,
 'accGoosePerShare': 196205211163,
 'depositFeeBP': 2000,
 'maxDepositAmount': 50000000000000000000000,
 'currentDepositAmount': 49999962582687227962551}

In [12]:
## Execution
# Mainnet (ChainID 0x38, 56 in decimal)

pool_id = 1
amount_send = w3.toWei(min(balance_egg, available_deposit), 'ether')
do_deposit = inc_contract.functions.deposit(pool_id, amount_send).buildTransaction({'chainId': 56, 'gas':250000, 'nonce': w3.eth.getTransactionCount(robot_addr)})

# Signing and sending
signed_txn = w3.eth.account.signTransaction(do_deposit, robot_account)
txn_hash = w3.eth.sendRawTransaction(signed_txn.rawTransaction)
txn_hash

HexBytes('0x18286eef65c316e4079de38dd9c4f33760a1e5066089548dcf6e94e856d2b7ae')

In [37]:
## Approve contract

# Before first deposit into pool we have to approve contract to transfer EGG (or whatever token) on behalf of our address
# Convention is to approve the maximum possible amount for uint256 which is more easily done in hex

approve_amount = 0xffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffff # w3.toWei(float(1.157920892373162e+59), 'ether')

approve = egg_contract.functions.approve(incubator_addr, approve_amount).buildTransaction({'chainId': 56, 'gas':100000, 'nonce': w3.eth.getTransactionCount(robot_addr)})

# Signing and sending
signed_txn = w3.eth.account.signTransaction(approve, robot_account)
txn_hash = w3.eth.sendRawTransaction(signed_txn.rawTransaction)
txn_hash

HexBytes('0x0d04b0f69f31a4955cd29d3de920c2ab7785a3c693ce313322a21b3863082d6c')